In [0]:
pip install jinja2

In [0]:
from jinja2 import Template

In [0]:
parameters = [
    {
        "table": "spotify_catalog.silver.factstream",
        "alias": "factstream",
        "cols" : ["factstream.stream_id", "factstream.listen_duration"]
    },
    {
        "table" : "spotify_catalog.silver.dimuser",
        "alias" : "dimuser",
        "cols" : ["dimuser.user_id","dimuser.user_name"],
        "condition" : "factstream.user_id = dimuser.user_id"
    },
    {
        "table" : "spotify_catalog.silver.dimtrack",
        "alias" : "dimtrack",
        "cols" : ["dimtrack.track_id","dimtrack.track_name"],
        "condition" : "factstream.track_id = dimtrack.track_id"
    }
]

In [0]:
query_text = """
SELECT
    {% for param in parameters %}
        {{ param.cols | join(', ') }}
        {% if not loop.last %},{% endif %}
    {% endfor %}
FROM
    {{ parameters[0].table }} AS {{ parameters[0].alias }}
    {% for param in parameters[1:] %}
        LEFT JOIN {{ param.table }} AS {{ param.alias }}
        ON {{ param.condition }}
    {% endfor %}
"""

template = Template(query_text)
final_query = template.render(parameters=parameters)

print(final_query)

In [0]:
display(spark.sql(final_query))